# NYC Airbnb Data Cleaning

 This notebook performs data cleaning on the NYC Airbnb dataset (`AB_NYC_2019.csv`).

 It includes the following steps:
 - Removing duplicates
 - Handling missing values
 - Fixing data types
 - Removing invalid entries (like zero or negative prices)



In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("AB_NYC_2019 (1).csv")
df.head()


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [ ]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [ ]:
df.shape

(48895, 16)

In [ ]:
df.isnull().sum()

,0
id,0
name,16
host_id,0
host_name,21
neighbourhood_group,0
neighbourhood,0
latitude,0
longitude,0
room_type,0
price,0


In [ ]:
# Drop duplicate rows
df_cleaned = df.drop_duplicates()


In [ ]:
# Drop rows where 'name' or 'host_name' is missing
df_cleaned = df_cleaned.dropna(subset=['name', 'host_name'])

In [ ]:
# Fill missing 'reviews_per_month' with 0
df_cleaned['reviews_per_month'] = df_cleaned['reviews_per_month'].fillna(0)

In [ ]:
# Fill missing 'last_review' with a placeholder date
df_cleaned['last_review'] = df_cleaned['last_review'].fillna('2000-01-01')


In [ ]:
df_cleaned.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


In [ ]:
# Convert 'last_review' to datetime format
df_cleaned['last_review'] = pd.to_datetime(df_cleaned['last_review'])


In [ ]:
df_cleaned.head(2)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355


In [ ]:
# Remove listings with price <= 0
df_cleaned = df_cleaned[df_cleaned['price'] > 0]

In [ ]:
# Remove listings with minimum_nights more than 365
df_cleaned = df_cleaned[df_cleaned['minimum_nights'] <= 365]

In [ ]:
# Check if 'id' and 'host_id' are unique
print("Duplicate IDs:", df_cleaned['id'].duplicated().sum())
print("Duplicate Host IDs:", df_cleaned['host_id'].duplicated().sum())


Duplicate IDs: 0
Duplicate Host IDs: 11423


In [ ]:
# Check if latitudes and longitudes are within NYC bounds
valid_lat = df_cleaned['latitude'].between(40.49, 40.92)
valid_lon = df_cleaned['longitude'].between(-74.25, -73.70)

# Filter only valid geo-locations
df_cleaned = df_cleaned[valid_lat & valid_lon]
print("Remaining rows after location filter:", len(df_cleaned))


Remaining rows after location filter: 48834


In [ ]:
# Ensure only expected room types are present
valid_room_types = ['Entire home/apt', 'Private room', 'Shared room', 'Hotel room']
df_cleaned = df_cleaned[df_cleaned['room_type'].isin(valid_room_types)]


In [ ]:
# Check data types and nulls again
print(df_cleaned.dtypes)
print(df_cleaned.isnull().sum())


id                                         int64
name                                      object
host_id                                    int64
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object
id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood         

In [ ]:
# Standardize string/text columns: strip spaces and lowercase
df_cleaned['name'] = df_cleaned['name'].str.strip().str.lower()
df_cleaned['host_name'] = df_cleaned['host_name'].str.strip().str.lower()
df_cleaned['neighbourhood_group'] = df_cleaned['neighbourhood_group'].str.strip().str.title()
df_cleaned['neighbourhood'] = df_cleaned['neighbourhood'].str.strip().str.title()
df_cleaned['room_type'] = df_cleaned['room_type'].str.strip().str.title()


In [ ]:
# If price needs to be scaled to 1000s for ML, you can normalize
# Example: price_in_k = price / 1000
df_cleaned['price_k'] = df_cleaned['price'] / 1000


In [ ]:
# Ensure last_review is in standard YYYY-MM-DD format
df_cleaned['last_review'] = pd.to_datetime(df_cleaned['last_review']).dt.date


## Outlier Detection and Treatment

We will detect and handle outliers using IQR (Interquartile Range) method for numeric columns.


In [ ]:
# IQR-based outlier removal function
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    print(f"Removing outliers in {column} outside range ({lower:.2f}, {upper:.2f})")
    return df[(df[column] >= lower) & (df[column] <= upper)]


In [ ]:
# Apply to selected numeric columns
df_cleaned = remove_outliers(df_cleaned, 'price')
df_cleaned = remove_outliers(df_cleaned, 'minimum_nights')
df_cleaned = remove_outliers(df_cleaned, 'reviews_per_month')


Removing outliers in price outside range (-90.00, 334.00)
Removing outliers in minimum_nights outside range (-5.00, 11.00)
Removing outliers in reviews_per_month outside range (-2.66, 4.62)


In [ ]:
df_cleaned.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price_k
0,2539,clean & quiet apt home by the park,2787,john,Brooklyn,Kensington,40.64749,-73.97237,Private Room,149,1,9,2018-10-19,0.21,6,365,0.149
1,2595,skylit midtown castle,2845,jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire Home/Apt,225,1,45,2019-05-21,0.38,2,355,0.225
2,3647,the village of harlem....new york !,4632,elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private Room,150,3,0,2000-01-01,0.00,1,365,0.150
4,5022,entire apt: spacious studio/loft by central park,7192,laura,Manhattan,East Harlem,40.79851,-73.94399,Entire Home/Apt,80,10,9,2018-11-19,0.10,1,0,0.080
5,5099,large cozy 1 br apartment in midtown east,7322,chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire Home/Apt,200,3,74,2019-06-22,0.59,1,129,0.200


In [ ]:
# Save after outlier removal
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned.to_csv("AB_NYC_2019_cleaned_final_outliers_removed.csv", index=False)
df_cleaned.describe()


,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_k
count,3.777600e+04,3.777600e+04,37776.000000,37776.000000,37776.000000,37776.000000,37776.000000,37776.000000,37776.000000,37776.000000,37776.000000
mean,1.851351e+07,6.368157e+07,40.727438,-73.950024,119.668467,2.760562,23.060991,0.977543,3.115047,95.073671,0.119668
std,1.083464e+07,7.596516e+07,0.056242,0.046631,67.313846,1.879150,40.885058,1.177922,20.244192,124.552207,0.067314
min,2.539000e+03,2.571000e+03,40.499790,-74.244420,10.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.010000
25%,9.132724e+06,7.459819e+06,40.688080,-73.981130,67.000000,1.000000,1.000000,0.060000,1.000000,0.000000,0.067000
50%,1.905704e+07,2.866495e+07,40.719920,-73.953610,100.000000,2.000000,6.000000,0.440000,1.000000,19.000000,0.100000
75%,2.817223e+07,9.593767e+07,40.763092,-73.933327,155.000000,3.000000,25.000000,1.590000,2.000000,177.000000,0.155000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,334.000000,11.000000,458.000000,4.610000,327.000000,365.000000,0.334000
